In [4]:
import os
import zipfile

# Create the datasets directory if it doesn't exist
if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Download the dataset using Kaggle API
!kaggle datasets download -d lexset/synthetic-asl-alphabet

# Extract the dataset
with zipfile.ZipFile('/content/synthetic-asl-alphabet.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("Dataset downloaded and extracted to 'dataset' directory.")

Dataset URL: https://www.kaggle.com/datasets/lexset/synthetic-asl-alphabet
License(s): copyright-authors
synthetic-asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset downloaded and extracted to 'dataset' directory.


In [5]:
# prompt: mount the drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    'dataset/Train_Alphabet',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    'dataset/Test_Alphabet',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 24300 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.


In [8]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'B', 'Blank', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [9]:
# CNN Model
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
# output layer
model.add(Dense(27, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [11]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [13]:
train_generator.samples

24300

In [14]:
steps_per_epoch=train_generator.samples // batch_size,
print(steps_per_epoch)

(189,)


In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
189/189 [==============================] - 1154s 6s/step - loss: 2.9511 - accuracy: 0.1213 - val_loss: 1.7660 - val_accuracy: 0.4375
Epoch 2/10
189/189 [==============================] - 1154s 6s/step - loss: 1.3963 - accuracy: 0.5361 - val_loss: 0.7474 - val_accuracy: 0.7656
Epoch 3/10
189/189 [==============================] - 1163s 6s/step - loss: 0.7586 - accuracy: 0.7578 - val_loss: 0.4122 - val_accuracy: 0.8664
Epoch 4/10
189/189 [==============================] - 1203s 6s/step - loss: 0.4598 - accuracy: 0.8521 - val_loss: 0.2655 - val_accuracy: 0.9129
Epoch 5/10
189/189 [==============================] - 1211s 6s/step - loss: 0.3294 - accuracy: 0.8956 - val_loss: 0.2056 - val_accuracy: 0.9282
Epoch 6/10
189/189 [==============================] - 1160s 6s/step - loss: 0.2535 - accuracy: 0.9175 - val_loss: 0.1784 - val_accuracy: 0.9353
Epoch 7/10
189/189 [==============================] - 1198s 6s/step - loss: 0.1955 - accuracy: 0.9375 - val_loss: 0.1683 - val_accuracy:

In [18]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Colab Notebooks/Trained model/model.json",'w') as json_file:
     json_file.write(model_json)
model.save("/content/drive/MyDrive/Colab Notebooks/Trained model/signlanguagedetectionmodel.h5")

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test accuracy: {test_accuracy}')

NameError: name 'model' is not defined